In [3]:
import pandas as pd
import numpy as np
import pickle
root="data/"

In [4]:
train_sample = pickle.load(open(root+"nlfl_train_sample_v3.pickle", "rb"))
val_sample = pickle.load(open(root+"nlfl_val_sample_v3.pickle", "rb"))
test_sample = pickle.load(open(root+"nlfl_test_sample_v3.pickle", "rb"))

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=1000, min_df=1, stop_words=[])

In [7]:
X_bong_train = np.asarray(tfidf.fit_transform(train_sample["A"].apply(str)).todense())
X_bong_test = np.asarray(tfidf.transform(test_sample["A"].apply(str)).todense())
X_bong_val = np.asarray(tfidf.transform(val_sample["A"].apply(str)).todense())

In [1]:
bong_nllf_train = pd.concat((train_sample, pd.DataFrame(X_bong_train, columns=tfidf.get_feature_names())), axis=1)
bong_nllf_test = pd.concat((test_sample, pd.DataFrame(X_bong_test, columns=tfidf.get_feature_names())), axis=1)
bong_nllf_val = pd.concat((val_sample, pd.DataFrame(X_bong_val, columns=tfidf.get_feature_names())), axis=1)

NameError: name 'pd' is not defined

In [2]:
from sklearn.tree import DecisionTreeClassifier

cols_nlfl = [c for c in bong_nllf_train.columns if "chatgpt_" in c or c in ["id", "label_Q", "label"]]
cols = cols_nlfl + tfidf.get_feature_names()

new_cols = [c for c in cols if c not in ["id", "label", "label_Q", "chatgpt_label"]] 
new_cols

NameError: name 'bong_nllf_train' is not defined

In [10]:
len(new_cols)

1144

In [11]:
new_best_features = ['chatgpt_v5 (N)', 'chatgpt_v6 (N)', 'chatgpt_v5_4 (N)',
       'chatgpt_v5_5 (N)', 'chatgpt_v6_2 (N)', 'chatgpt_Q3_1 (Y)',
       'chatgpt_Q3_2 (Y)', 'chatgpt_Q3_4 (Y)', 'chatgpt_Q3_7 (Y)',
       'chatgpt_Q3_9 (N)', 'chatgpt_Q3_9 (Y)', 'chatgpt_Q3_10 (Y)',
       'chatgpt_Q3_11 (N)', 'chatgpt_Q3_11 (Y)', 'chatgpt_Q3_12 (Y)',
       'chatgpt_Q3_13 (Y)', 'chatgpt_Q3_14 (N)', 'chatgpt_Q3_14 (Y)',
       'chatgpt_Q3_15 (N)', 'chatgpt_Q3_16 (Y)', 'chatgpt_Q3_7_e2 (Y)',
       'chatgpt_Q3_7_e3 (Y)', 'chatgpt_Q3_7_e4 (Y)', 'chatgpt_10_e1 (Y)',
       'chatgpt_10_e4 (Y)', 'chatgpt_7_e3 (Y)', 'chatgpt_7_e4 (N)',
       'chatgpt_7_e4 (Y)', 'chatgpt_7_e5 (Y)', 'chatgpt_Q3_4_e1 (N)',
       'chatgpt_Q3_4_e5 (Y)', 'chatgpt_Q3_5_e1 (N)',
       'chatgpt_Q3_5_e1 (Y)', 'chatgpt_Q3_5_e2 (Y)',
       'chatgpt_Q3_5_e3 (Y)', 'chatgpt_HF_1 (N)', 'chatgpt_HF_3 (Y)',
       'chatgpt_HF_5 (Y)', 'chatgpt_HF_6 (N)', 'chatgpt_SP_1 (Y)',
       'chatgpt_SP_4 (Y)', 'chatgpt_SP_6 (N)'] + [
 'asi',
 'bien porque',
 'corecto',
 'correcta',
 'creo que',
 'entonces',
 'entre',
 'equibocada',
 'equivocado',
 'este', #
 'falta',
 'incorrecto',
 'malo',
 'mide', #
 'ninguna',
 'ninguno',
 'no', #
 'nose',
 'porque', #
 'puede',
 'que', #
 'quedan',
 'quedo',
 'rason',
 'tiene', #
 'ya que',
 'yo creo'
 ]

In [12]:
clf = DecisionTreeClassifier(random_state=42, min_impurity_decrease=0.0012, max_depth=5, criterion="gini")
clf.fit(bong_nllf_train[new_best_features], bong_nllf_train["label"])

DecisionTreeClassifier(max_depth=5, min_impurity_decrease=0.0012,
                       random_state=42)

In [13]:
from matplotlib import pyplot as plt
from sklearn import tree

In [14]:
fig, ax = plt.subplots(1, 1, figsize=(265, 85))
tree.plot_tree(clf, feature_names=new_best_features, ax=ax, fontsize=45, class_names=["C0","C1"]);

In [15]:
from sklearn.metrics import classification_report

In [16]:
print(classification_report(bong_nllf_train["label"], clf.predict(bong_nllf_train[new_best_features]), digits=4))

              precision    recall  f1-score   support

         0.0     0.9217    0.9574    0.9392     10019
         1.0     0.6293    0.4708    0.5386      1540

    accuracy                         0.8926     11559
   macro avg     0.7755    0.7141    0.7389     11559
weighted avg     0.8827    0.8926    0.8858     11559



In [17]:
val_sub_index = bong_nllf_val.index
print(classification_report(bong_nllf_val.loc[val_sub_index]["label"], clf.predict(bong_nllf_val.loc[val_sub_index][new_best_features]), digits=4))

              precision    recall  f1-score   support

           0     0.9207    0.9570    0.9385      2510
           1     0.6263    0.4665    0.5347       388

    accuracy                         0.8913      2898
   macro avg     0.7735    0.7117    0.7366      2898
weighted avg     0.8812    0.8913    0.8844      2898



In [19]:
sub_index_test = bong_nllf_test.index#[bong_nllf_test["label_Q"] == 3].index
print(classification_report(bong_nllf_test.loc[sub_index_test]["label"], clf.predict(bong_nllf_test.loc[sub_index_test][new_best_features]), digits=4))

              precision    recall  f1-score   support

           0     0.9315    0.9298    0.9306       541
           1     0.7226    0.7279    0.7253       136

    accuracy                         0.8892       677
   macro avg     0.8271    0.8289    0.8279       677
weighted avg     0.8895    0.8892    0.8894       677

